<a href="https://colab.research.google.com/github/sysadminamit/CusomWordEmbeddingsNLP/blob/master/CustomWordEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz

--2019-07-04 06:06:26--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz’

reviews_Electronics 100%[===================>] 472.88M  1.36MB/s    in 4m 4s   

2019-07-04 06:10:30 (1.93 MB/s) - ‘reviews_Electronics_5.json.gz’ saved [495854086/495854086]



In [0]:
import pandas as pd
import os
fn = 'reviews_Electronics_5.json.gz'
print(os.path.isfile(fn))
df = pd.read_json(fn, lines=True, compression='gzip')
df.head()

True


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200
1,0528881469,"[12, 15]",1,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
2,0528881469,"[43, 45]",3,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400
3,0528881469,"[9, 10]",2,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1290556800
4,0528881469,"[0, 0]",1,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1317254400


In [0]:
#data = df.to_csv('ElectronicsProductReviewCSV.csv')
import pandas as pd
dataset = pd.read_csv('ElectronicsProductReviewCSV.csv')
dataset.shape

(1689188, 10)

In [0]:
dataset = dataset.dropna()

In [0]:
import numpy as np
remove_n = 1000000

drop_indices = np.random.choice(dataset.index, remove_n, replace=False)
dataset = dataset.drop(drop_indices)

In [0]:
dataset.shape

(663229, 10)

In [0]:
dataset[['helpulobtained','totalhelpfulscore']] = pd.DataFrame(dataset['helpful'].str.split().values.tolist())

In [0]:
dataset["helpfulobtained"] = dataset['helpulobtained'].str.replace('[^\w\s]','')
dataset["totalhelpfulScore"] = dataset['totalhelpfulscore'].str.replace('[^\w\s]','')

In [0]:
dataset = dataset.drop(['helpulobtained', 'totalhelpfulscore'], axis=1)

In [0]:
dataset['helpfulobtained'] = pd.to_numeric(dataset['helpfulobtained'], errors='coerce')
dataset = dataset.dropna(subset=['helpfulobtained'])
dataset['helpfulobtained'] = dataset['helpfulobtained'].astype(int)

In [0]:
dataset['totalhelpfulScore'] = pd.to_numeric(dataset['totalhelpfulScore'], errors='coerce')
dataset = dataset.dropna(subset=['totalhelpfulScore'])
dataset['totalhelpfulScore'] = dataset['totalhelpfulScore'].astype(int)

In [0]:
dataset['Helpfulfactor'] = dataset['helpfulobtained']/dataset['totalhelpfulScore']

In [0]:
dataset['Helpfulfactor'] = dataset['Helpfulfactor'].fillna(0)
dataset.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,helpfulobtained,totalhelpfulScore,Helpfulfactor
3,3,0528881469,"[9, 10]",2,"Not going to write a long review, even thought...","11 24, 2010",A1H8PY3QHMQQA0,"Dave M. Shaw ""mack dave""","Great grafics, POOR GPS",1290556800,2,2,1.000000
4,4,0528881469,"[0, 0]",1,I've had mine for a year and here's what we go...,"09 29, 2011",A24EV6RXELQZ63,Wayne Smith,"Major issues, only excuses for support",1317254400,0,0,0.000000
9,9,0594451647,"[3, 3]",5,This product really works great but I found th...,"01 20, 2014",A3BY5KCNQZXV5U,Matenai,This works great but read the details...,1390176000,7,9,0.777778
10,10,0594481813,"[2, 2]",4,This item is just as was described in the orig...,"04 16, 2014",A7S2B0I67WNWB,AllyMG,As expected,1397606400,0,0,0.000000
15,15,0594481813,"[0, 0]",3,It does 2A and charges a DEAD Nook in a few ho...,"08 31, 2013",AT09WGFUM934H,John,This is the oem charger you get with the nook hd+,1377907200,0,0,0.000000


In [0]:
import gensim, logging, os, re, string, tensorflow
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from gensim.utils import simple_preprocess

print('gensim version: \t%s'     % gensim.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)

gensim version: 	3.6.0
TensorFlow version: 	1.14.0


In [0]:
dataset['review_Model']= dataset['reviewText']+ " " + dataset['summary']

In [0]:
df = dataset[['reviewerID','reviewerName','review_Model']]
 #Select features from original dataset to form a new dataframe 

In [0]:
# For each row, combine all the columns into one column
df1 = df.apply(lambda x: ','.join(x.astype(str)), axis=1)

In [0]:
# Store them in a pandas dataframe
df_clean = pd.DataFrame({'clean': df1})

In [0]:
# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [0]:
# show the example of list of list format of the custom corpus for gensim modeling 
sent[:2]

[['A1H8PY3QHMQQA0',
  'Dave M. Shaw "mack dave"',
  'Not going to write a long review',
  " even thought this unit deserves one. I've driven well over 1-mil miles and done most of my own routing so I pretty know whats the fastest and shortest. Have been using a basic garmin for the past three years and with ANY Gps unit they'll ALL get you in trouble if you let them. I was really excited about this unit",
  ' due to the size and the features. Allot of great grafics and on screen info thats usefull. But the most basic item that it was lacking was the gps tracking. I gave this unit allot of leadway on its mistakes due to the fact that it had allot of cool stuff that it did',
  " but its ability to track you and route you was not even close to what the basic garmin could due. Its like the prossesor that they installed in the TND 700 was 10 years old. Example if I needed to make a simple route change I.E. in town down to the next street due to the fact that I couldn't make the turn or the 

In [0]:
model = gensim.models.Word2Vec(sent, min_count=1,size= 300,workers=3, window =3, sg = 1)

In [0]:
model.most_similar('A1H8PY3QHMQQA0')[:5]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('I received this DF820A3 as a gift at Christmas', 0.3288156986236572),
 (' flexible qualities and become more rigid', 0.27521437406539917),
 (' had 2 67mm filters in their cases', 0.27266764640808105),
 ('Felipe V. Aburaya', 0.2705111503601074),
 (" but I've since figured out how to have it pop back out while still in my ear (pull back).Once in",
  0.2664572596549988)]

In [0]:
MODEL_DIR = 'emb_data/'

In [0]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
model.save(os.path.join(MODEL_DIR,'word2vec'))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
weights     = model.wv.vectors
index_words = model.wv.index2word

vocab_size    = weights.shape[0]
embedding_dim = weights.shape[1]

print('Shape of weights:', weights.shape)
print('Vocabulary size: %i' % vocab_size)
print('Embedding size: %i'  % embedding_dim)

with open(os.path.join(MODEL_DIR,'metadata.tsv'), 'w') as f:
    f.writelines("\n".join(index_words))

# Required if you re-run without restarting the kernel
tf.reset_default_graph()
    
W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]), trainable=False, name="W")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])

embedding_init = W.assign(embedding_placeholder)
writer = tf.summary.FileWriter(MODEL_DIR, graph=tf.get_default_graph())
saver = tf.train.Saver()

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = W.name
embedding.metadata_path = './metadata.tsv'
projector.visualize_embeddings(writer, config)

with tf.Session() as sess:
    sess.run(embedding_init, feed_dict={embedding_placeholder: weights})
    save_path = saver.save(sess, os.path.join(MODEL_DIR, "model.cpkt"))

Shape of weights: (1566749, 300)
Vocabulary size: 1566749
Embedding size: 300
